In [1]:
#import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.framework import ops

ops.reset_default_graph()
#sess = tf.Session()

In [2]:
setwd = 'C:\\Users\\PraveenPC\\Desktop\\Zekelabs\\Projects\\ForestFires\\'

In [3]:
rawfile = pd.read_csv(setwd + 'forestfires.csv')

In [4]:
le = LabelEncoder()
rawfile['month'] = le.fit_transform(rawfile.month)
rawfile['day'] = le.fit_transform(rawfile.day)

In [5]:
feature_cols = list(set(rawfile.columns) - set(['area']))
#type(feature_cols) #list

In [6]:
finalraw = rawfile[feature_cols]
finalrawtarget = rawfile['area']
#type(finalraw) #pandas.core.frame.DataFrame

In [57]:
finalraw = finalraw.as_matrix()
#type(finalraw) #numpy.ndarray

array([[   8.2,    7. ,    0. , ...,   94.3,    0. ,   86.2],
       [  18. ,    7. ,    0. , ...,  669.1,    5. ,   90.6],
       [  14.6,    7. ,    0. , ...,  686.9,    2. ,   90.6],
       ..., 
       [  21.2,    7. ,    0. , ...,  665.6,    3. ,   81.6],
       [  25.6,    1. ,    0. , ...,  614.7,    2. ,   94.4],
       [  11.8,    6. ,    0. , ...,  106.7,    5. ,   79.5]])

In [8]:
train_indices = np.random.choice(len(finalraw), round(len(finalraw)*0.8), replace=False)
test_indices = np.array(list(set(range(len(finalraw))) - set(train_indices)))

In [9]:
x_vals_train = finalraw[train_indices] # 80% of feature columns data
x_vals_test = finalraw[test_indices] # 20% of feature columns data
y_vals_train = finalrawtarget[train_indices] # 80% of target column data
y_vals_test = finalrawtarget[test_indices] # 20% of target column data

In [10]:
batch_size = 50
#none = we are not sure of no.of rows, 12 = no.of features
x_placeholder_featuredata = tf.placeholder(shape=[None,12],dtype=tf.float32) 
#none = we are not sure of no.of rows, 1 = target column
y_placeholder_targetdata = tf.placeholder(shape = [None,1],dtype=tf.float32)


In [11]:
#hidden layers
hidden_value_nodes = 15
A1 = tf.Variable(tf.random_normal(shape=[12,hidden_value_nodes]))
b1 = tf.Variable(tf.random_normal(shape=[hidden_value_nodes]))
                 
A2 = tf.Variable(tf.random_normal(shape=[hidden_value_nodes,1]))
b2 = tf.Variable(tf.random_normal(shape=[1]))

In [12]:
#Applying ReLU activation function to each layer.
hidden_output = tf.nn.relu(tf.add(tf.matmul(x_placeholder_featuredata,A1),b1))
final_output = tf.nn.relu(tf.add(tf.matmul(hidden_output,A2),b2))

In [13]:
#loss function
loss = tf.reduce_mean(tf.square(y_placeholder_targetdata - final_output))

In [14]:
#init = tf.global_variables_initializer()
#sess.run(init)   

In [15]:
gd = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [16]:
prediction = tf.round(final_output)

prediction_correct = tf.cast(tf.equal(prediction, y_placeholder_targetdata), tf.float32)

accuracy = tf.reduce_mean(prediction_correct)

In [17]:
loss_vec = []
train_acc = []
test_acc = []

In [21]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in range(50):
        rand_index = np.random.choice(len(x_vals_train), size=batch_size)
        rand_x = x_vals_train[rand_index]
        rand_y = np.transpose([y_vals_train[rand_index]])
        
        train_process = sess.run(gd, feed_dict={x_placeholder_featuredata: rand_x, y_placeholder_targetdata: rand_y})

        temp_loss = sess.run(loss, feed_dict={x_placeholder_featuredata: rand_x, y_placeholder_targetdata: rand_y})

        loss_vec.append(temp_loss)

        

In [55]:
x_vals_train[rand_index]

array([[  1.18000000e+01,   6.00000000e+00,   0.00000000e+00,
          4.50000000e+00,   9.00000000e+00,   1.10000000e+00,
          3.00000000e+00,   3.00000000e+00,   3.10000000e+01,
          1.06700000e+02,   5.00000000e+00,   7.95000000e+01],
       [  2.38000000e+01,   2.00000000e+00,   0.00000000e+00,
          1.80000000e+00,   5.00000000e+00,   5.60000000e+00,
          8.47000000e+01,   5.00000000e+00,   5.10000000e+01,
          3.76600000e+02,   2.00000000e+00,   9.08000000e+01],
       [  1.90000000e+01,   7.00000000e+00,   0.00000000e+00,
          5.80000000e+00,   1.10000000e+01,   1.22000000e+01,
          1.17900000e+02,   4.00000000e+00,   3.40000000e+01,
          6.68000000e+02,   0.00000000e+00,   9.24000000e+01],
       [  1.88000000e+01,   2.00000000e+00,   0.00000000e+00,
          4.90000000e+00,   1.00000000e+00,   7.70000000e+00,
          7.53000000e+01,   2.00000000e+00,   3.50000000e+01,
          4.66600000e+02,   4.00000000e+00,   9.30000000e+01],
    